In [372]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [373]:
df = pd.read_excel('/content/flight_price.xlsx')
df.head(2)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662


In [374]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [375]:
df.shape

(10683, 11)

In [376]:
df.describe()

,Price
count,10683.000000
mean,9087.064121
std,4611.359167
min,1759.000000
25%,5277.000000
50%,8372.000000
75%,12373.000000
max,79512.000000


**Dropping Duplicates**

In [377]:
df.drop_duplicates(inplace=True)

In [378]:
df['Additional_Info'].value_counts()

,count
Additional_Info,
No info,8183
In-flight meal not included,1926
No check-in baggage included,318
1 Long layover,19
Change airports,7
Business class,4
No Info,3
1 Short layover,1
Red-eye flight,1


**Fixing date columns and converting it to numerical**

In [379]:
df['Day'] = df['Date_of_Journey'].str.split('/').str[0].astype(int)
df['Month'] = df['Date_of_Journey'].str.split('/').str[1].astype(int)
df['Year'] = df['Date_of_Journey'].str.split('/').str[2].astype(int)

**Fixing duration columns and converting it to numerical**

In [380]:
# Extract hours and minutes as numbers, missing ones become 0
df['Duration_hours'] = df['Duration'].str.extract(r'(\d+)h').fillna(0).astype(int)
df['Duration_minutes'] = df['Duration'].str.extract(r'(\d+)m').fillna(0).astype(int)


In [381]:
df['Duration_minutes'] =  df['Duration_minutes'].fillna(0)

In [382]:
df['Duration_hours'] = df['Duration_hours'].astype(int)
df['Duration_hours'] = df['Duration_minutes'].astype(int)

**Fixing Arrival columns and converting it to numerical**

In [383]:
df['Arrival_hours'] = df['Arrival_Time'].str.split(' ').str[0].str.split(':').str[0].astype(int)
df['Arrival_minutes'] = df['Arrival_Time'].str.split(' ').str[0].str.split(':').str[1].astype(int)

**Fixing Dep_Time columns and converting it to numerical**

In [384]:
df['Dep_hours'] = df['Dep_Time'].str.split(':').str[0].astype(int)
df['Dep_Min'] = df['Dep_Time'].str.split(':').str[1].astype(int)

**Encoding Total Stops to numerical values**

In [385]:
mod_val = df['Total_Stops'].mode()[0]
mod_val

df['Total_Stops'] = df['Total_Stops'].map({'non-stop' : 0, '2 stops' : 2, '1 stop' : 1, '3 stops' : 3, np.nan : int(mod_val[0]), '4 stops': 4})

**Encoding the Airline,	Source,	Destination columns**

In [386]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

encoded_array = pd.DataFrame(ohe.fit_transform(df[['Airline','Source','Destination']]), columns = ohe.get_feature_names_out())

df = pd.concat([df, encoded_array], axis=1).reset_index(drop=True)


In [387]:
encoded_additonalInfo = pd.DataFrame(ohe.fit_transform(df[['Additional_Info']]), columns=ohe.get_feature_names_out())

df = pd.concat([df, encoded_additonalInfo], axis=1).reset_index(drop=True)

**Dropping duplicate / fixed columns**

In [388]:
df.drop(['Date_of_Journey', 'Additional_Info', 'Dep_Time', 'Duration', 'Arrival_Time', 'Route', 'Airline',	'Source',	'Destination'], axis=1, inplace=True)

In [389]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10671 entries, 0 to 10670
Data columns (total 45 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Total_Stops                                   10463 non-null  float64
 1   Price                                         10463 non-null  float64
 2   Day                                           10463 non-null  float64
 3   Month                                         10463 non-null  float64
 4   Year                                          10463 non-null  float64
 5   Duration_hours                                10463 non-null  float64
 6   Duration_minutes                              10463 non-null  float64
 7   Arrival_hours                                 10463 non-null  float64
 8   Arrival_minutes                               10463 non-null  float64
 9   Dep_hours                                     10463 non-null 

In [390]:
df.head(2)

,Total_Stops,Price,Day,Month,Year,Duration_hours,Duration_minutes,Arrival_hours,Arrival_minutes,Dep_hours,...,Additional_Info_1 Short layover,Additional_Info_2 Long layover,Additional_Info_Business class,Additional_Info_Change airports,Additional_Info_In-flight meal not included,Additional_Info_No Info,Additional_Info_No check-in baggage included,Additional_Info_No info,Additional_Info_Red-eye flight,Additional_Info_nan
0,0.0,3897.0,24.0,3.0,2019.0,50.0,50.0,1.0,10.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2.0,7662.0,1.0,5.0,2019.0,25.0,25.0,13.0,15.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


**Final Cleaned Data**

In [392]:
df.to_csv("Flight_Price_Cleaned.csv", index=False)